In [1]:
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 도구 정의
@tool
def multiply(num1: int|float, num2: int|float) -> int|float:
    """입력받은 두 수를 곱한 결과를 반환한다. """
    return num1 * num2

@tool
def plus(num1: int|float, num2: int|float) -> int|float:
    """입력 받은 두 수를 더한 결과를 반환한다. """
    return num1 + num2

@tool
def minus(num1: int|float, num2: int|float) -> int|float:
    """입력 받은 두 수를 뺀 결과를 반환한다. """
    return num1 - num2

@tool
def divide(num1: int|float, num2: int|float) -> int|float:
    """입력 받은 두 수를 나눈 결과를 반환한다. """
    return num1 / num2


In [4]:
agent_prompt = ChatPromptTemplate(
    [
        ("system", dedent("""
            당신은 계산기 agent입니다. 사칙연산을 할 수있는 toolkit 을 가지고 있습니다. 
            두개의 숫자를 사칙연산하는 작업은 toolkit의 plus, minus, multiply, divide 툴들을 이용합니다.
            연산이 복잡할 경우 단계적으로 나눠서 계산하도록 합니다.
            """)
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


toolkit = [multiply, plus, minus, divide]
model = ChatOpenAI(model="gpt-4o")
agent = create_tool_calling_agent(model, toolkit, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=toolkit)

In [5]:
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """
    store(dict) 에서 session_id의 InMemoryChatMessageHistory를 찾아서 반환.
    store에 session_id의 InMemoryChatMessageHistory가 없다면 생성하여 반환한다.
    invoke() 시 전달한 config의 session_id를 파라미터로 받는다.

    Parameter
        session_id: 반환할 Memory 객체의 id
    return
        (InMemoryChatMessageHistory: BaseChatMessageHistory 타입 객체.)
    """
    # 위 dictionary에서 찾아서 InMemoryChatMessageHistory 반환.
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    runnable=agent_executor,                     # agent를 runnable로 등록
    get_session_history=get_session_history,  # 메모리 저장 객체를 반환하는 함수. 
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
input_message = input("Question:")
while True:
    if input_message == "!q":
        break
    res =  agent_with_chat_history.invoke({"input":input_message}, config={"configurable":{"session_id": "test-id"}})
    print(res['output'])
    input_message = input("Question:")

1 + 1 = 2
2에 2를 더하면 4입니다.
1에 무엇을 더하거나 빼거나 곱하거나 나누고 싶으신가요? 추가적인 계산이 필요하다면 말씀해 주세요!
안내가 필요하시면 언제든 말씀해 주세요! 도움이 필요하신가요?


In [6]:
# Stream 처리
# callback 정의
from langchain.callbacks.base import BaseCallbackHandler
class StreamHandler(BaseCallbackHandler):
    def __init__(self, initial_text=""):
        self.full_text = [initial_text]

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        """token을 받을 때 마다 호출됨"""
        self.full_text.append(token)
        print(token, end="")


input_message = input("Question:")
callback = StreamHandler()
while True:
    if input_message == "!q":
        break
    res =  agent_with_chat_history.invoke({"input":input_message}, config={"configurable":{"session_id": "test-id"}, "callbacks":[callback]})
    print(res['output'])
    input_message = input("Question:")

1 + 1 = 2입니다.1 + 1 = 2입니다.
1 + 1의 결과인 2에서 5를 빼면 -3입니다.1 + 1의 결과인 2에서 5를 빼면 -3입니다.
